In [1]:
#import dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
pd.set_option('display.max_columns', 200)
import re
import collections
import MySQLdb as sql

In [2]:
#read in data from ?fao.csv? and display head for initial examination
csv_file = "../Resources/fao.csv"
fao_data_df = pd.read_csv(csv_file, encoding="ANSI")
fao_data_df.head()

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,1928.0,1904.0,1666.0,1950.0,2001.0,1808.0,2053.0,2045.0,2154.0,1819.0,1963.0,2215.0,2310.0,2335.0,2434.0,2512.0,2282.0,2454.0,2443.0,2129.0,2133.0,2068.0,1994.0,1851.0,1791.0,1683.0,2194.0,1801.0,1754.0,1640.0,1539.0,1582.0,1840.0,1855.0,1853.0,2177.0,2343.0,2407.0,2463.0,2600.0,2668.0,2776.0,3095.0,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,183.0,183.0,182.0,220.0,220.0,195.0,231.0,235.0,238.0,213.0,205.0,233.0,246.0,246.0,255.0,263.0,235.0,254.0,270.0,259.0,248.0,217.0,217.0,197.0,186.0,200.0,193.0,202.0,191.0,199.0,197.0,249.0,218.0,260.0,319.0,254.0,326.0,347.0,270.0,372.0,411.0,448.0,460.0,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,76.0,76.0,76.0,76.0,76.0,75.0,71.0,72.0,73.0,74.0,71.0,70.0,72.0,76.0,77.0,80.0,60.0,65.0,64.0,64.0,60.0,55.0,53.0,51.0,48.0,46.0,46.0,47.0,46.0,43.0,43.0,40.0,50.0,46.0,41.0,44.0,50.0,48.0,43.0,26.0,29.0,70.0,48.0,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,237.0,237.0,237.0,238.0,238.0,237.0,225.0,227.0,230.0,234.0,223.0,219.0,225.0,240.0,244.0,255.0,185.0,203.0,198.0,202.0,189.0,174.0,167.0,160.0,151.0,145.0,145.0,148.0,145.0,135.0,132.0,120.0,155.0,143.0,125.0,138.0,159.0,154.0,141.0,84.0,83.0,122.0,144.0,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,210.0,210.0,214.0,216.0,216.0,216.0,235.0,232.0,236.0,200.0,201.0,216.0,228.0,231.0,234.0,240.0,228.0,234.0,228.0,226.0,210.0,199.0,192.0,182.0,173.0,170.0,154.0,148.0,137.0,144.0,126.0,90.0,141.0,150.0,159.0,108.0,90.0,99.0,72.0,35.0,48.0,89.0,63.0,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200


In [3]:
# insert an id column and rename column 'Item' to 'FAO_Category' for later merge
# https://stackoverflow.com/questions/38862293/how-to-add-incremental-numbers-to-a-new-column-using-pandas/38862389

fao_data_df.insert(0, 'ID', range(0, 0 + len(fao_data_df)))
fao_data_df = fao_data_df.rename(columns={'Item':'FAO_Category'})
fao_data_df

,ID,Area Abbreviation,Area Code,Area,Item Code,FAO_Category,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,1928.0,1904.0,1666.0,1950.0,2001.0,1808.0,2053.0,2045.0,2154.0,1819.0,1963.0,2215.0,2310.0,2335.0,2434.0,2512.0,2282.0,2454.0,2443.0,2129.0,2133.0,2068.0,1994.0,1851.0,1791.0,1683.0,2194.0,1801.0,1754.0,1640.0,1539.0,1582.0,1840.0,1855.0,1853.0,2177.0,2343.0,2407.0,2463.0,2600.0,2668.0,2776.0,3095.0,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,183.0,183.0,182.0,220.0,220.0,195.0,231.0,235.0,238.0,213.0,205.0,233.0,246.0,246.0,255.0,263.0,235.0,254.0,270.0,259.0,248.0,217.0,217.0,197.0,186.0,200.0,193.0,202.0,191.0,199.0,197.0,249.0,218.0,260.0,319.0,254.0,326.0,347.0,270.0,372.0,411.0,448.0,460.0,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,76.0,76.0,76.0,76.0,76.0,75.0,71.0,72.0,73.0,74.0,71.0,70.0,72.0,76.0,77.0,80.0,60.0,65.0,64.0,64.0,60.0,55.0,53.0,51.0,48.0,46.0,46.0,47.0,46.0,43.0,43.0,40.0,50.0,46.0,41.0,44.0,50.0,48.0,43.0,26.0,29.0,70.0,48.0,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,237.0,237.0,237.0,238.0,238.0,237.0,225.0,227.0,230.0,234.0,223.0,219.0,225.0,240.0,244.0,255.0,185.0,203.0,198.0,202.0,189.0,174.0,167.0,160.0,151.0,145.0,145.0,148.0,145.0,135.0,132.0,120.0,155.0,143.0,125.0,138.0,159.0,154.0,141.0,84.0,83.0,122.0,144.0,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,210.0,210.0,214.0,216.0,216.0,216.0,235.0,232.0,236.0,200.0,201.0,216.0,228.0,231.0,234.0,240.0,228.0,234.0,228.0,226.0,210.0,199.0,192.0,182.0,173.0,170.0,154.0,148.0,137.0,144.0,126.0,90.0,141.0,150.0,159.0,108.0,90.0,99.0,72.0,35.0,48.0,89.0,63.0,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200
5,5,AFG,2,Afghanistan,2514,Maize and products,5142,Food,1000 tonnes,33.94,67.71,403.0,403.0,410.0,415.0,415.0,413.0,454.0,448.0,455.0,383.0,386.0,416.0,439.0,445.0,451.0,463.0,439.0,451.0,440.0,437.0,407.0,384.0,371.0,353.0,334.0,330.0,298.0,287.0,265.0,279.0,245.0,170.0,272.0,289.0,310.0,209.0,173.0,192.0,141.0,66.0,93.0,170.0,117.0,231.0,67.0,82.0,67.0,69.0,71.0,82.0,73.0,77,76
6,6,AFG,2,Afghanistan,2517,Millet and products,5142,Food,1000 tonnes,33.94,67.71,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,26.0,27.0,27.0,28.0,29.0,37.0,32.0,33.0,31.0,31.0,29.0,27.0,28.0,26.0,25.0,23.0,23.0,23.0,23.0,24.0,24.0,18.0,22.0,20.0,21.0,17.0,20.0,21.0,17.0,20.0,20.0,18.0,16.0,15.0,21.0,11.0,19.0,21.0,18.0,14.0,14.0,14,12
7,7,AFG,2,Afghanistan,2520,"Cereals, Other",5142,Food,1000 tonnes,33.94,67.71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0
8,8,AFG,2,Afghanistan,2531,Potatoes and products,5142,Food,1000 tonnes,33.94,67.71,111.0,97.0,103.0,110.0,113.0,117.0,128.0,130.0,134.0,125.0,147.0,138.0,143.0,160.0,169.0,324.0,176.0,225.0,232.0,240.0,247.0,248.0,242.0,235.0,226.0,217.0,196.0,198.0,184.0,205.0,203.0,210.0,210.0,211.0,212.0,213.0,214.0,214.0,217.0,219.0,215.0,217.0,347.0,276.0,294.0,294.0,260.0,242.0,250.0,192.0,169.0,196,230
9,9,AFG,2,Afghanistan,2536,Sugar cane,5521,Feed,1000 tonnes,33.94,67.71,45.0,45.0,45.0,45.0,31.0,14.0,19.0,30.0,34.0,15.0,0.0,0.0,28.0,32.0,20.0,28.0,24.0,24.0,34.0,61.0,50.0,43.0,38.0,46.0,23.0,25.0,3.

In [4]:
#create a list from a unique set of the 'Item' categories from initial dataset
# create a dataframe to be exported to csv
FAO_ctgry_df = pd.DataFrame({'FAO_Category': list(set(fao_data_df['FAO_Category']))})
FAO_ctgry_df.to_csv('./../FAO_ctgry_df.csv')
FAO_ctgry_df.head()

,FAO_Category
0,Alcoholic Beverages
1,Eggs
2,Wheat and products
3,Sugar beet
4,Cottonseed Oil


Analyze categories and create column of normalized category 'bins' in excel

In [5]:
#read category datagrame with new column set index to normalized category
csv_file = "./../FAO_norm_ctgry_df.csv"
FAO_nrm_ctgry_df = pd.read_csv(csv_file)
#FAO_nrm_ctgry_df = FAO_nrm_ctgry_df.set_index('normalized_category')
FAO_nrm_ctgry_df.head()

,normalized_category,FAO_Category
0,Oil,"Oilcrops, Other"
1,Grain,"Pulses, Other and products"
2,Spice,Pepper
3,Grain,Cereals - Excluding Beer
4,Plant,Cassava and products


In [6]:
# merge to add normalized category to FAO
fao_norm_df = pd.merge(fao_data_df, FAO_nrm_ctgry_df, on='FAO_Category', how='left')
fao_norm_df

,ID,Area Abbreviation,Area Code,Area,Item Code,FAO_Category,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,normalized_category
0,0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,1928.0,1904.0,1666.0,1950.0,2001.0,1808.0,2053.0,2045.0,2154.0,1819.0,1963.0,2215.0,2310.0,2335.0,2434.0,2512.0,2282.0,2454.0,2443.0,2129.0,2133.0,2068.0,1994.0,1851.0,1791.0,1683.0,2194.0,1801.0,1754.0,1640.0,1539.0,1582.0,1840.0,1855.0,1853.0,2177.0,2343.0,2407.0,2463.0,2600.0,2668.0,2776.0,3095.0,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895,Grain
1,1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,183.0,183.0,182.0,220.0,220.0,195.0,231.0,235.0,238.0,213.0,205.0,233.0,246.0,246.0,255.0,263.0,235.0,254.0,270.0,259.0,248.0,217.0,217.0,197.0,186.0,200.0,193.0,202.0,191.0,199.0,197.0,249.0,218.0,260.0,319.0,254.0,326.0,347.0,270.0,372.0,411.0,448.0,460.0,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422,Grain
2,2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,76.0,76.0,76.0,76.0,76.0,75.0,71.0,72.0,73.0,74.0,71.0,70.0,72.0,76.0,77.0,80.0,60.0,65.0,64.0,64.0,60.0,55.0,53.0,51.0,48.0,46.0,46.0,47.0,46.0,43.0,43.0,40.0,50.0,46.0,41.0,44.0,50.0,48.0,43.0,26.0,29.0,70.0,48.0,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360,Grain
3,3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,237.0,237.0,237.0,238.0,238.0,237.0,225.0,227.0,230.0,234.0,223.0,219.0,225.0,240.0,244.0,255.0,185.0,203.0,198.0,202.0,189.0,174.0,167.0,160.0,151.0,145.0,145.0,148.0,145.0,135.0,132.0,120.0,155.0,143.0,125.0,138.0,159.0,154.0,141.0,84.0,83.0,122.0,144.0,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89,Grain
4,4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,210.0,210.0,214.0,216.0,216.0,216.0,235.0,232.0,236.0,200.0,201.0,216.0,228.0,231.0,234.0,240.0,228.0,234.0,228.0,226.0,210.0,199.0,192.0,182.0,173.0,170.0,154.0,148.0,137.0,144.0,126.0,90.0,141.0,150.0,159.0,108.0,90.0,99.0,72.0,35.0,48.0,89.0,63.0,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200,Grain
5,5,AFG,2,Afghanistan,2514,Maize and products,5142,Food,1000 tonnes,33.94,67.71,403.0,403.0,410.0,415.0,415.0,413.0,454.0,448.0,455.0,383.0,386.0,416.0,439.0,445.0,451.0,463.0,439.0,451.0,440.0,437.0,407.0,384.0,371.0,353.0,334.0,330.0,298.0,287.0,265.0,279.0,245.0,170.0,272.0,289.0,310.0,209.0,173.0,192.0,141.0,66.0,93.0,170.0,117.0,231.0,67.0,82.0,67.0,69.0,71.0,82.0,73.0,77,76,Grain
6,6,AFG,2,Afghanistan,2517,Millet and products,5142,Food,1000 tonnes,33.94,67.71,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,26.0,27.0,27.0,28.0,29.0,37.0,32.0,33.0,31.0,31.0,29.0,27.0,28.0,26.0,25.0,23.0,23.0,23.0,23.0,24.0,24.0,18.0,22.0,20.0,21.0,17.0,20.0,21.0,17.0,20.0,20.0,18.0,16.0,15.0,21.0,11.0,19.0,21.0,18.0,14.0,14.0,14,12,Grain
7,7,AFG,2,Afghanistan,2520,"Cereals, Other",5142,Food,1000 tonnes,33.94,67.71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,Alcohol
8,8,AFG,2,Afghanistan,2531,Potatoes and products,5142,Food,1000 tonnes,33.94,67.71,111.0,97.0,103.0,110.0,113.0,117.0,128.0,130.0,134.0,125.0,147.0,138.0,143.0,160.0,169.0,324.0,176.0,225.0,232.0,240.0,247.0,248.0,242.0,235.0,226.0,217.0,196.0,198.0,184.0,205.0,203.0,210.0,210.0,211.0,212.0,213.0,214.0,214.0,217.0,219.0,215.0,217.0,347.0,276.0,294.0,294.0,260.0,242.0,250.0,192.0,169.0,196,230,Plant
9,9,AFG,2,Afghanistan,2536,Sugar cane,5521,Feed,1000 tonnes,33.94,67.71,45.0,45.0,45.0,45.0,31.0,14.0,19.0,30.0,34.0,15.0,0.0,

In [7]:
#import second databaes low_memory=False option is way to show non-truncated columns

csv_file = "../Resources/en.openfoodfacts.org.products.tsv"
ofd_data_df = pd.read_csv(csv_file, sep='\t', low_memory=False)
ofd_data_df.head()


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,0000000003087,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:FR,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000004530,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,United States,"Bananas, vegetable oil (coconut oil, cor

In [8]:
# remove the columns with multirow text as initial cleanup to the massive open food database
mini_food_df = ofd_data_df[ofd_data_df.columns.difference(['ingredients_text', 'states', 'states_tags', 'states_en','additives', 'additives_en', 'additives_n' ])]


In [9]:
# OFD with ingredients for potential categorizations ordered by column with fewest NaN
food_ing_df = ofd_data_df[ofd_data_df.columns.difference(['states', 'states_tags', 'states_en','additives', 'additives_en', 'additives_n' ])]
food_ing_df = food_ing_df[['ingredients_text', 'created_t', 'url', 'creator', 'created_datetime', 'code',
       'countries_en', 'countries_tags', 'countries', 'product_name', 'brands',
       'brands_tags', 'energy_100g', 'proteins_100g', 'salt_100g',
       'sodium_100g', 'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_from_palm_oil_n', 'fat_100g', 'sugars_100g',
       'carbohydrates_100g', 'saturated-fat_100g', 'nutrition-score-fr_100g',
       'nutrition_grade_fr', 'nutrition-score-uk_100g', 'fiber_100g',
       'serving_size', 'additives_tags', 'cholesterol_100g', 'trans-fat_100g',
       'calcium_100g', 'vitamin-c_100g', 'iron_100g', 'vitamin-a_100g',
       'pnns_groups_2', 'pnns_groups_1', 'quantity', 'categories_tags',
       'main_category_en', 'main_category', 'categories', 'categories_en',
       'packaging_tags', 'packaging', 'image_url', 'image_small_url',
       'purchase_places', 'labels_en', 'labels', 'labels_tags', 'generic_name',
       'stores', 'manufacturing_places', 'manufacturing_places_tags',
       'allergens']].copy()
#food_ing_df = food_df.dropna(axis=0, subset=['categories_en'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['energy_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['proteins_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['carbohydrates_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['saturated-fat_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['fiber_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['fat_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['sugars_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['ingredients_text'])
food_ing_df.columns

Index(['ingredients_text', 'created_t', 'url', 'creator', 'created_datetime',
       'code', 'countries_en', 'countries_tags', 'countries', 'product_name',
       'brands', 'brands_tags', 'energy_100g', 'proteins_100g', 'salt_100g',
       'sodium_100g', 'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_from_palm_oil_n', 'fat_100g', 'sugars_100g',
       'carbohydrates_100g', 'saturated-fat_100g', 'nutrition-score-fr_100g',
       'nutrition_grade_fr', 'nutrition-score-uk_100g', 'fiber_100g',
       'serving_size', 'additives_tags', 'cholesterol_100g', 'trans-fat_100g',
       'calcium_100g', 'vitamin-c_100g', 'iron_100g', 'vitamin-a_100g',
       'pnns_groups_2', 'pnns_groups_1', 'quantity', 'categories_tags',
       'main_category_en', 'main_category', 'categories', 'categories_en',
       'packaging_tags', 'packaging', 'image_url', 'image_small_url',
       'purchase_places', 'labels_en', 'labels', 'labels_tags', 'generic_name',
       'stores', 'manufacturing_places', 

In [10]:
# create dataframe from ingredients list and pare down to main ingredient (first) and remove non-english characters using regex
# write pared ingredient list to csv
OFD_ing_df = pd.DataFrame({'OFD_Ingredients': list(set(food_ing_df['ingredients_text']))})
#OFD_ctgry_df['en_boolean'] = OFD_ctgry_df['OFD_Categories'].apply(lambda x: 'True' if re.search(r'en:', x) == True else "False")
OFD_ing_df['Main_Ingredient'] = OFD_ing_df['OFD_Ingredients'].str.split(',').str[0]
OFD_ing_df = OFD_ing_df[~OFD_ing_df['Main_Ingredient'].astype(str).str.contains(r'[^A-Za-z]', regex=True)]
#OFD_ing_df
OFD_ing_df['Main_Ingredient'].to_csv('./../OFD_ing_df.csv')
OFD_Ing_List = list(OFD_ing_df['Main_Ingredient'])

In [11]:
# define function to determine percentage of missing values in each column
# https://stackoverflow.com/questions/26266362/how-to-count-the-nan-values-in-a-column-in-pandas-dataframe

def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [12]:
# run reduced dataframe through function to ascertain which columns contain lots of empty values
# sort values in ascending order so cleanest columns are at the top
miss_df = missing_values_table(mini_food_df)
miss_df = miss_df.sort_values('% of Total Values', ascending=True)
miss_df.head() 

Your selected dataframe has 156 columns.
There are 154 columns that have missing values.


,Missing Values,% of Total Values
created_t,3,0.0
url,26,0.0
creator,3,0.0
created_datetime,10,0.0
code,26,0.0


In [13]:
# limit missing data df to columns with at least 90% of total values
# print index (list of columns) to screen for copy/paste
miss_df = miss_df.loc[miss_df['% of Total Values'] < 90, :]
miss_df.index

Index(['created_t', 'url', 'creator', 'created_datetime', 'code',
       'countries_en', 'countries_tags', 'countries', 'product_name', 'brands',
       'brands_tags', 'energy_100g', 'proteins_100g', 'salt_100g',
       'sodium_100g', 'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_from_palm_oil_n', 'fat_100g', 'sugars_100g',
       'carbohydrates_100g', 'saturated-fat_100g', 'nutrition-score-fr_100g',
       'nutrition_grade_fr', 'nutrition-score-uk_100g', 'fiber_100g',
       'serving_size', 'additives_tags', 'cholesterol_100g', 'trans-fat_100g',
       'calcium_100g', 'vitamin-c_100g', 'iron_100g', 'vitamin-a_100g',
       'pnns_groups_2', 'pnns_groups_1', 'quantity', 'categories_tags',
       'main_category_en', 'main_category', 'categories', 'categories_en',
       'packaging_tags', 'packaging', 'image_url', 'image_small_url',
       'purchase_places', 'labels_en', 'labels', 'labels_tags', 'generic_name',
       'stores', 'manufacturing_places', 'manufacturing_place

In [14]:
# create new dataframe with only desired columns in desired order with the good old copy-paste

ofd_rel_df = ofd_data_df[['created_t', 'url', 'creator', 'created_datetime', 'code',
       'countries_en', 'countries_tags', 'countries', 'product_name', 'brands',
       'brands_tags', 'energy_100g', 'proteins_100g', 'salt_100g',
       'sodium_100g', 'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_from_palm_oil_n', 'fat_100g', 'sugars_100g',
       'carbohydrates_100g', 'saturated-fat_100g', 'nutrition-score-fr_100g',
       'nutrition_grade_fr', 'nutrition-score-uk_100g', 'fiber_100g',
       'serving_size', 'additives_tags', 'cholesterol_100g', 'trans-fat_100g',
       'calcium_100g', 'vitamin-c_100g', 'iron_100g', 'vitamin-a_100g',
       'pnns_groups_2', 'pnns_groups_1', 'quantity', 'categories_tags',
       'main_category_en', 'main_category', 'categories', 'categories_en',
       'packaging_tags', 'packaging', 'image_url', 'image_small_url',
       'purchase_places', 'labels_en', 'labels', 'labels_tags', 'generic_name',
       'stores', 'manufacturing_places', 'manufacturing_places_tags',
       'allergens']].copy()
ofd_rel_df.head()

,created_t,url,creator,created_datetime,code,countries_en,countries_tags,countries,product_name,brands,brands_tags,energy_100g,proteins_100g,salt_100g,sodium_100g,ingredients_that_may_be_from_palm_oil_n,ingredients_from_palm_oil_n,fat_100g,sugars_100g,carbohydrates_100g,saturated-fat_100g,nutrition-score-fr_100g,nutrition_grade_fr,nutrition-score-uk_100g,fiber_100g,serving_size,additives_tags,cholesterol_100g,trans-fat_100g,calcium_100g,vitamin-c_100g,iron_100g,vitamin-a_100g,pnns_groups_2,pnns_groups_1,quantity,categories_tags,main_category_en,main_category,categories,categories_en,packaging_tags,packaging,image_url,image_small_url,purchase_places,labels_en,labels,labels_tags,generic_name,stores,manufacturing_places,manufacturing_places_tags,allergens
0,1474103866,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,2016-09-17T09:17:46Z,0000000003087,France,en:france,en:FR,Farine de blé noir,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1489069957,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09T14:32:37Z,0000000004530,United States,en:united-states,US,Banana Chips Sweetened (Whole),NaN,NaN,2243.0,3.57,0.00000,0.000,0.0,0.0,28.57,14.29,64.29,28.57,14.0,d,14.0,3.6,28 g (1 ONZ),NaN,0.018,0.0,0.000,0.0214,0.00129,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1489069957,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09T14:32:37Z,0000000004559,United States,en:united-states,US,Peanuts,Torn & Glasser,torn-glasser,1941.0,17.86,0.63500,0.250,0.0,0.0,17.86,17.86,60.71,0.00,0.0,b,0.0,7.1,28 g (0.25 cup),NaN,0.000,0.0,0.071,0.0000,0.00129,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1489055731,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09T10:35:31Z,0000000016087,United States,en:united-states,US,Organic Salted Nut Mix,Grizzlies,grizzlies,2540.0,17.86,1.22428,0.482,0.0,0.0,57.14,3.57,17.86,5.36,12.0,d,12.0,7.1,28 g (0.25 cup),NaN,NaN,NaN,0.143,NaN,0.00514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1489055653,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09T10:34:13Z,0000000016094,United States,en:united-states,US,Organic Polenta,Bob's Red Mill,bob-s-red-mill,1552.0,8.57,NaN,NaN,0.0,0.0,1.43,NaN,77.14,NaN,NaN,NaN,NaN,5.7,35 g (0.25 cup),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# remove all rows with NaN values in relevant columns
relevant_cats= ['categories_en', 'energy_100g', 'proteins_100g', 'carbohydrates_100g', 
                'fat_100g', 'saturated-fat_100g', 'fiber_100g', 'sugars_100g']
clean_df = ofd_rel_df.dropna(axis=0, subset=['categories_en'])
clean_df = clean_df.dropna(axis=0, subset=['energy_100g'])
clean_df = clean_df.dropna(axis=0, subset=['proteins_100g'])
clean_df = clean_df.dropna(axis=0, subset=['carbohydrates_100g'])
clean_df = clean_df.dropna(axis=0, subset=['saturated-fat_100g'])
clean_df = clean_df.dropna(axis=0, subset=['fiber_100g'])
clean_df = clean_df.dropna(axis=0, subset=['fat_100g'])
clean_df = clean_df.dropna(axis=0, subset=['sugars_100g'])
clean_df.head()


,created_t,url,creator,created_datetime,code,countries_en,countries_tags,countries,product_name,brands,brands_tags,energy_100g,proteins_100g,salt_100g,sodium_100g,ingredients_that_may_be_from_palm_oil_n,ingredients_from_palm_oil_n,fat_100g,sugars_100g,carbohydrates_100g,saturated-fat_100g,nutrition-score-fr_100g,nutrition_grade_fr,nutrition-score-uk_100g,fiber_100g,serving_size,additives_tags,cholesterol_100g,trans-fat_100g,calcium_100g,vitamin-c_100g,iron_100g,vitamin-a_100g,pnns_groups_2,pnns_groups_1,quantity,categories_tags,main_category_en,main_category,categories,categories_en,packaging_tags,packaging,image_url,image_small_url,purchase_places,labels_en,labels,labels_tags,generic_name,stores,manufacturing_places,manufacturing_places_tags,allergens
176,1484605978,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-16T22:32:58Z,0000000290616,Canada,en:canada,Canada,Salade Cesar,Kirkland Signature,kirkland-signature,1210.0,22.00,2.1600,0.8500,0.0,0.0,12.00,0.000,23.00,7.000,6.0,c,6.0,2.000,100 g,NaN,0.05600,NaN,0.0000,0.000000,0.000000,0.000000,Vegetables,Fruits and vegetables,0.980 kg,"en:plant-based-foods-and-beverages,en:plant-ba...",Plant-based foods and beverages,en:plant-based-foods-and-beverages,Légumes-feuilles,"Plant-based foods and beverages,Plant-based fo...",frais,Frais,NaN,NaN,NaN,NaN,NaN,NaN,Laitue en salade,Costco,Brossard Québec,brossard-quebec,NaN
182,1484501528,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-15T17:32:08Z,0000001938067,Canada,en:canada,Canada,Chaussons tressés aux pommes,Kirkland Signature,kirkland-signature,1090.0,3.33,0.6470,0.2550,0.0,0.0,10.70,24.700,38.70,2.000,9.0,c,9.0,2.000,150 g / 1 chausson,"en:e202,en:e211,en:e330,en:e509,en:e920",0.00533,0.667,0.0133,0.001600,0.000480,0.000000,Biscuits and cakes,Sugary snacks,1.200 kg,"en:sugary-snacks,en:biscuits-and-cakes,en:past...",Sugary snacks,en:sugary-snacks,"Snacks sucrés,Biscuits et gâteaux,Pâtisseries","Sugary snacks,Biscuits and cakes,Pastries",frais,Frais,NaN,NaN,Brossard Québec,NaN,NaN,NaN,Pâtisserie,Costco,Brossard Québec,brossard-quebec,NaN
183,1488464896,http://world-en.openfoodfacts.org/product/0000...,b7,2017-03-02T14:28:16Z,0000004302544,Canada,en:canada,Canada,Pain Burger Artisan,Kirkland Signature,kirkland-signature,1160.0,10.00,1.5200,0.6000,0.0,0.0,1.11,1.110,53.30,0.333,1.0,b,1.0,2.220,90 g / 1 pain,NaN,0.00000,0.000,0.0222,0.000000,0.006000,0.000000,unknown,unknown,1.008 kg / 12 pain,fr:boulange,fr:Boulange,fr:boulange,Boulange,fr:Boulange,"frais,plastique","Frais,plastique",NaN,NaN,"Brossard,Québec",NaN,NaN,NaN,Pain,Costco,"Brossard,Québec","brossard,quebec",NaN
185,1420150193,http://world-en.openfoodfacts.org/product/0000...,teolemon,2015-01-01T22:09:53Z,0000007020254,France,en:france,France,Root Beer,A&W,a-w,215.0,0.00,0.0616,0.0242,0.0,0.0,0.00,13.600,14.20,0.000,18.0,e,3.0,0.000,33 cl,"en:e150,en:e211,en:e999",NaN,NaN,NaN,NaN,NaN,NaN,Sweetened beverages,Beverages,33 cl e,"en:beverages,en:carbonated-drinks,en:sodas,en:...",Beverages,en:beverages,"Boissons,Boissons gazeuses,Sodas,Boissons sucr...","Beverages,Carbonated drinks,Sodas,Sugared beve...","canette,metal","Canette,Métal",http://en.openfoodfacts.org/images/products/00...,http://en.openfoodfacts.org/images/products/00...,France,"Kosher,fr:Contient-des-ogm","Kascher,Contient des OGM","en:kosher,fr:contient-des-ogm",NaN,NaN,États-Unis,etats-unis,NaN
187,1491943307,http://world-en.openfoodfacts.org/product/0000...,b7,2017-04-11T20:41:47Z,0000008237798,Canada,en:canada,Canada,Quiche Lorraine,Kirkland Signature,kirkland-signature,478.0,5.36,0.4990,0.1960,0.0,0.0,6.79,0.714,7.86,2.860,2.0,b,2.0,0.357,280 g / 1/5 de la Quiche,"en:e235,en:e316,en:e407",0.01070,0.000,0.0286,0.000429,0.000643,0.000011,Pizza pies and quiche,Composite foods,1 400 kg,"en:meals,en:pizzas-pies-and-quiches,en:quiches...",Meals,en:meals,Quiches lorraines,"Meals,Pizzas pies and quiches,Quiches,fr:Quich...",frai,Frai,NaN,NaN,"Brossard,Québec",NaN,NaN,NaN,Quiche lorraine,Cost

In [16]:
#create category dataframe from relevant clean_df and append a boolean column stating if category is in english
# 3,640 categoryies in original df, 1,287 after removing non-english; 1463 categories in clean_df, 483 after non-english filter
OFD_ctgry_df = pd.DataFrame({'main_category': list(set(clean_df['main_category']))})
#OFD_ctgry_df['en_boolean'] = OFD_ctgry_df['OFD_Categories'].apply(lambda x: 'True' if re.search(r'en:', x) == True else "False")
OFD_ctgry_df['en_boolean'] = OFD_ctgry_df['main_category'].str.match(r'en:')
OFD_ctgry_df.head()

,main_category,en_boolean
0,pl:soupes-deshydratees,False
1,fr:cakes-aux-legumes,False
2,pt:lacticineos,False
3,fr:tartiflette,False
4,fr:feuilles-de-brick,False


In [17]:
# insert an id column
# https://stackoverflow.com/questions/38862293/how-to-add-incremental-numbers-to-a-new-column-using-pandas/38862389

OFD_ctgry_df.insert(0, 'ID', range(0, 0 + len(OFD_ctgry_df)))

OFD_ctgry_df.head()

,ID,main_category,en_boolean
0,0,pl:soupes-deshydratees,False
1,1,fr:cakes-aux-legumes,False
2,2,pt:lacticineos,False
3,3,fr:tartiflette,False
4,4,fr:feuilles-de-brick,False


In [18]:
# reduce dataframe to only english (supposedly, lol) categories and strip the prefix
# make list from column
OFD_en_ctgry_df = OFD_ctgry_df.loc[OFD_ctgry_df['en_boolean'] == True, :].reset_index().drop(columns=['index', 'en_boolean'])
OFD_en_ctgry_df['main_category'] = OFD_en_ctgry_df['main_category'].str.slice(3)
OFD_en_ctgry_df = OFD_en_ctgry_df.rename(columns={'main_category':'OFD_Category'})
OFD_en_ctgry_df.to_csv('./../OFD_en_ctgry_df.csv')

#OFD_Categories = list(OFD_en_ctgry_df['OFD_Category'])
#OFD_Categories

OFD_en_ctgry_df.head()

,ID,OFD_Category
0,5,chocolate-mints
1,6,graines
2,7,artisan-products
3,11,fruit-juices-and-nectars
4,14,pates-a-tartiner-aux-noisettes-et-au-cacao


In [19]:
#read category datagrame with new column and option to set index to normalized category
csv_file = "./../OFD_en_norm_ctgry_df.csv"
OFD_nrm_ctgry_df = pd.read_csv(csv_file)
#OFD_nrm_ctgry_df = OFD_nrm_ctgry_df.set_index('normalized_category')
OFD_nrm_ctgry_df.head()


,normalized_category,OFD_Category
0,Dry_Goods,psyllium-husk
1,Meat,chicken-thighs
2,Dry_Goods,snacks
3,Dry_Goods,salted-snack
4,Meat,chorizos


In [20]:
# compare original (no reset index) df to compare the two category lists which will become primary keys to links the databases
csv_file = "./../OFD_en_norm_ctgry_df.csv"
OFD_nrm_ctgry_df = pd.read_csv(csv_file)
ofd_nrm_ctgry = list(OFD_nrm_ctgry_df['normalized_category'])
set(ofd_nrm_ctgry)


{'Coconut',
 'Dairy',
 'Dry_Goods',
 'Fruit',
 'Grain',
 'Meat',
 'Oil',
 'Other',
 'Palm',
 'Plant',
 'Seafood',
 'Seed',
 'Soy',
 'Spice',
 'Sweet',
 'Vegetable'}

In [21]:
# compare original (no reset index) df to compare the two category lists which will become primary keys to links the databases
csv_file = "./../FAO_norm_ctgry_df.csv"
FAO_nrm_ctgry_df = pd.read_csv(csv_file)
fao_nrm_ctgry = list(FAO_nrm_ctgry_df['normalized_category'])
set(fao_nrm_ctgry)

{'Alcohol',
 'Coconut',
 'Dairy',
 'Dry_Goods',
 'Fruit',
 'Grain',
 'Infant',
 'Meat',
 'Oil',
 'Other',
 'Palm',
 'Plant',
 'Seafood',
 'Seed',
 'Soy',
 'Spice',
 'Sweet',
 'Vegetable'}

In [22]:
# merge to get ID
ofd_norm1_df = pd.merge(OFD_en_ctgry_df, OFD_nrm_ctgry_df, on='OFD_Category', how='left')
ofd_norm1_df.head()

,ID,OFD_Category,normalized_category
0,5,chocolate-mints,Sweet
1,6,graines,Grain
2,7,artisan-products,Other
3,11,fruit-juices-and-nectars,Fruit
4,14,pates-a-tartiner-aux-noisettes-et-au-cacao,Sweet


In [23]:
# merge to get original category column
ofd_norm2_df = pd.merge(OFD_ctgry_df, ofd_norm1_df, on='ID', how='left')
ofd_norm2_df.head()

,ID,main_category,en_boolean,OFD_Category,normalized_category
0,0,pl:soupes-deshydratees,False,NaN,NaN
1,1,fr:cakes-aux-legumes,False,NaN,NaN
2,2,pt:lacticineos,False,NaN,NaN
3,3,fr:tartiflette,False,NaN,NaN
4,4,fr:feuilles-de-brick,False,NaN,NaN


In [24]:
ofd_norm_df = ofd_norm2_df.dropna(axis=0, subset=['OFD_Category'])
ofd_norm_df.head()

,ID,main_category,en_boolean,OFD_Category,normalized_category
5,5,en:chocolate-mints,True,chocolate-mints,Sweet
6,6,en:graines,True,graines,Grain
7,7,en:artisan-products,True,artisan-products,Other
11,11,en:fruit-juices-and-nectars,True,fruit-juices-and-nectars,Fruit
14,14,en:pates-a-tartiner-aux-noisettes-et-au-cacao,True,pates-a-tartiner-aux-noisettes-et-au-cacao,Sweet


In [25]:
# remove rows with NaN values and drop en_boolean column
ofd_clean_norm_df = pd.merge(clean_df, ofd_norm_df, on='main_category', how='left')
ofd_clean_norm_df = ofd_clean_norm_df.dropna(axis=0, subset=['normalized_category']).drop(columns='en_boolean')

ofd_clean_norm_df.head()

,created_t,url,creator,created_datetime,code,countries_en,countries_tags,countries,product_name,brands,brands_tags,energy_100g,proteins_100g,salt_100g,sodium_100g,ingredients_that_may_be_from_palm_oil_n,ingredients_from_palm_oil_n,fat_100g,sugars_100g,carbohydrates_100g,saturated-fat_100g,nutrition-score-fr_100g,nutrition_grade_fr,nutrition-score-uk_100g,fiber_100g,serving_size,additives_tags,cholesterol_100g,trans-fat_100g,calcium_100g,vitamin-c_100g,iron_100g,vitamin-a_100g,pnns_groups_2,pnns_groups_1,quantity,categories_tags,main_category_en,main_category,categories,categories_en,packaging_tags,packaging,image_url,image_small_url,purchase_places,labels_en,labels,labels_tags,generic_name,stores,manufacturing_places,manufacturing_places_tags,allergens,ID,OFD_Category,normalized_category
0,1484605978,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-16T22:32:58Z,0000000290616,Canada,en:canada,Canada,Salade Cesar,Kirkland Signature,kirkland-signature,1210.0,22.00,2.1600,0.85000,0.0,0.0,12.00,0.000,23.00,7.00,6.0,c,6.0,2.000,100 g,NaN,0.05600,NaN,0.0000,0.000000,0.000000,0.000000,Vegetables,Fruits and vegetables,0.980 kg,"en:plant-based-foods-and-beverages,en:plant-ba...",Plant-based foods and beverages,en:plant-based-foods-and-beverages,Légumes-feuilles,"Plant-based foods and beverages,Plant-based fo...",frais,Frais,NaN,NaN,NaN,NaN,NaN,NaN,Laitue en salade,Costco,Brossard Québec,brossard-quebec,NaN,295.0,plant-based-foods-and-beverages,Other
1,1484501528,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-15T17:32:08Z,0000001938067,Canada,en:canada,Canada,Chaussons tressés aux pommes,Kirkland Signature,kirkland-signature,1090.0,3.33,0.6470,0.25500,0.0,0.0,10.70,24.700,38.70,2.00,9.0,c,9.0,2.000,150 g / 1 chausson,"en:e202,en:e211,en:e330,en:e509,en:e920",0.00533,0.667,0.0133,0.001600,0.000480,0.000000,Biscuits and cakes,Sugary snacks,1.200 kg,"en:sugary-snacks,en:biscuits-and-cakes,en:past...",Sugary snacks,en:sugary-snacks,"Snacks sucrés,Biscuits et gâteaux,Pâtisseries","Sugary snacks,Biscuits and cakes,Pastries",frais,Frais,NaN,NaN,Brossard Québec,NaN,NaN,NaN,Pâtisserie,Costco,Brossard Québec,brossard-quebec,NaN,761.0,sugary-snacks,Sweet
3,1420150193,http://world-en.openfoodfacts.org/product/0000...,teolemon,2015-01-01T22:09:53Z,0000007020254,France,en:france,France,Root Beer,A&W,a-w,215.0,0.00,0.0616,0.02420,0.0,0.0,0.00,13.600,14.20,0.00,18.0,e,3.0,0.000,33 cl,"en:e150,en:e211,en:e999",NaN,NaN,NaN,NaN,NaN,NaN,Sweetened beverages,Beverages,33 cl e,"en:beverages,en:carbonated-drinks,en:sodas,en:...",Beverages,en:beverages,"Boissons,Boissons gazeuses,Sodas,Boissons sucr...","Beverages,Carbonated drinks,Sodas,Sugared beve...","canette,metal","Canette,Métal",http://en.openfoodfacts.org/images/products/00...,http://en.openfoodfacts.org/images/products/00...,France,"Kosher,fr:Contient-des-ogm","Kascher,Contient des OGM","en:kosher,fr:contient-des-ogm",NaN,NaN,États-Unis,etats-unis,NaN,973.0,beverages,Other
4,1491943307,http://world-en.openfoodfacts.org/product/0000...,b7,2017-04-11T20:41:47Z,0000008237798,Canada,en:canada,Canada,Quiche Lorraine,Kirkland Signature,kirkland-signature,478.0,5.36,0.4990,0.19600,0.0,0.0,6.79,0.714,7.86,2.86,2.0,b,2.0,0.357,280 g / 1/5 de la Quiche,"en:e235,en:e316,en:e407",0.01070,0.000,0.0286,0.000429,0.000643,0.000011,Pizza pies and quiche,Composite foods,1 400 kg,"en:meals,en:pizzas-pies-and-quiches,en:quiches...",Meals,en:meals,Quiches lorraines,"Meals,Pizzas pies and quiches,Quiches,fr:Quich...",frai,Frai,NaN,NaN,"Brossard,Québec",NaN,NaN,NaN,Quiche lorraine,Costco,"Brossard,Québec","brossard,quebec",NaN,493.0,meals,Other
5,1418732959,http://world-en.openfoodfacts.org/product/0000...,tacinte,2014-12-16T12:29:19Z,0000027533024,United Kingdom,en:united-kingdom,United Kingdom,Luxury Christmas Pudding,"Asda,Asda Extra Special","asda,asda-extra-special",1284.0,3.90,0.2000,0.07874,0.0,0.0,7.00,44.000,54.20,3.60,10.0,c,10.0,4.700,113.5g,"en:e163,en:e170,en:e330,en:e375",NaN,NaN,NaN,Na

In [26]:
# export dataframes to be merged in mySQL for better performance
ofd_clean_norm_df.to_csv('./../ofd_clean_norm_df.csv')
fao_norm_df.to_csv('./../fao_norm_df.csv')



In [27]:
# connect to MySQL database and retrieve merged table limited to 1000 due to processing power
# https://stackoverflow.com/questions/10065051/python-pandas-and-databases-like-mysql
conn = sql.connect(host='127.0.0.1', port=3306, user='root', password='Password', db='food_data')
sqlquery = 'select * from fao_ofd_df;'
fao_ofd_df = pd.read_sql(sqlquery, con=conn)
fao_ofd_df.head()

,created_t,url,creator,created_datetime,code,countries_en,countries_tags,countries,product_name,brands,brands_tags,energy_100g,proteins_100g,salt_100g,sodium_100g,ingredients_that_may_be_from_palm_oil_n,ingredients_from_palm_oil_n,fat_100g,sugars_100g,carbohydrates_100g,saturated-fat_100g,nutrition-score-fr_100g,nutrition_grade_fr,nutrition-score-uk_100g,fiber_100g,serving_size,additives_tags,cholesterol_100g,trans-fat_100g,calcium_100g,vitamin-c_100g,iron_100g,vitamin-a_100g,pnns_groups_2,pnns_groups_1,quantity,categories_tags,main_category_en,main_category,categories,categories_en,packaging_tags,packaging,image_url,image_small_url,purchase_places,labels_en,labels,labels_tags,generic_name,stores,manufacturing_places,manufacturing_places_tags,allergens,OFD_ID,OFD_Category,FAO_ID,Area Abbreviation,Area Code,Area,Item Code,FAO_Category,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,normalized_category
0,1484605978,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-16T22:32:58Z,0000000290616,Canada,en:canada,Canada,Salade Cesar,Kirkland Signature,kirkland-signature,1210.0,22.00,2.160,0.850,0.0,0.0,12.0,0.0,23.0,7.0,6.0,c,6.0,2.0,100 g,,0.055999999999999994,,0.0,0.0,0.0,0.0,Vegetables,Fruits and vegetables,0.980 kg,"en:plant-based-foods-and-beverages,en:plant-ba...",Plant-based foods and beverages,en:plant-based-foods-and-beverages,Légumes-feuilles,"Plant-based foods and beverages,Plant-based fo...",frais,Frais,,,,,,,Laitue en salade,Costco,Brossard Québec,brossard-quebec,,208.0,plant-based-foods-and-beverages,82,AFG,2,Afghanistan,2928,Miscellaneous,5142,Food,1000 tonnes,33.94,67.71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2,2,Other
1,1484605978,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-16T22:32:58Z,0000000290616,Canada,en:canada,Canada,Salade Cesar,Kirkland Signature,kirkland-signature,1210.0,22.00,2.160,0.850,0.0,0.0,12.0,0.0,23.0,7.0,6.0,c,6.0,2.0,100 g,,0.055999999999999994,,0.0,0.0,0.0,0.0,Vegetables,Fruits and vegetables,0.980 kg,"en:plant-based-foods-and-beverages,en:plant-ba...",Plant-based foods and beverages,en:plant-based-foods-and-beverages,Légumes-feuilles,"Plant-based foods and beverages,Plant-based fo...",frais,Frais,,,,,,,Laitue en salade,Costco,Brossard Québec,brossard-quebec,,208.0,plant-based-foods-and-beverages,205,ALB,3,Albania,2928,Miscellaneous,5142,Food,1000 tonnes,41.15,20.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,Other
2,1484605978,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-16T22:32:58Z,0000000290616,Canada,en:canada,Canada,Salade Cesar,Kirkland Signature,kirkland-signature,1210.0,22.00,2.160,0.850,0.0,0.0,12.0,0.0,23.0,7.0,6.0,c,6.0,2.0,100 g,,0.055999999999999994,,0.0,0.0,0.0,0.0,Vegetables,Fruits and vegetables,0.980 kg,"en:plant-based-foods-and-beverages,en:plant-ba...",Plant-based foods and beverages,en:plant-based-foods-and-beverages,Légumes-feuilles,"Plant-based foods and beverages,Plant-based fo...",frais,Frais,,,,,,,Laitue en salade,Costco,Brossard Québec,brossard-quebec,,208.0,plant-based-foods-and-beverages,329,DZA,4,Algeria,2928,Miscellaneous,5142,Food,1000 tonnes,28.03,1.66,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,3.0,3.0,2.0,3.0,3.0,3.0,2.0,3.0,3.0,3.0,3.0,1.0,4.0,4.0,3.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,5.0,8.0,13.0,12.0,20.0,21,22,Other
3,1484605978,http://wo

In [28]:
# save final merge dataframe to csv
fao_ofd_df.to_csv('./../fao_ofd_df.csv)')

Perform aggregations on merged data

In [29]:
# sum of merged data frames grouped by normalized category
ctgry_grp_sum = fao_ofd_df.groupby('normalized_category').sum()
ctgry_grp_sum.head()

,created_t,energy_100g,proteins_100g,salt_100g,sodium_100g,ingredients_that_may_be_from_palm_oil_n,ingredients_from_palm_oil_n,fat_100g,sugars_100g,carbohydrates_100g,saturated-fat_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,fiber_100g,OFD_ID,FAO_ID,Area Code,Item Code,Element Code,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
normalized_category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Dairy,7443095245,8600.0,114.50,6.90000,2.716535,0.0,0.0,172.00,5.500,9.50,106.00,75.0,100.0,6.000,4715.0,297,10,14127,26468,169.70,338.55,598.0,602.0,673.0,688.0,756.0,789.0,856.0,897.0,905.0,784.0,715.0,742.0,824.0,867.0,887.0,913.0,834.0,847.0,858.0,886.0,911.0,906.0,963.0,938.0,760.0,513.0,583.0,602.0,620.0,879.0,935.0,964.0,1097.0,1307.0,1448.0,1654.0,1815.0,1900.0,2168.0,1751.0,1166.0,1991.0,1902.0,1926.0,1857.0,1772.0,1888.0,1850.0,1892.0,1949.0,1940.0,2176,2201
Grain,116341009902,90090.0,514.80,42.90000,16.889764,0.0,0.0,756.60,1177.800,3104.40,218.40,312.0,312.0,117.000,65520.0,16469,343,204464,413204,1890.74,1032.64,10765.0,10915.0,10422.0,11205.0,11591.0,11253.0,12321.0,12611.0,13060.0,12143.0,12519.0,13950.0,14424.0,14918.0,16031.0,16747.0,16445.0,17980.0,18389.0,18419.0,18674.0,19104.0,19097.0,19181.0,20034.0,21287.0,21859.0,21441.0,23260.0,22151.0,21508.0,22248.0,23507.0,24388.0,23974.0,24994.0,24959.0,26890.0,27618.0,26943.0,28492.0,30117.0,31331.0,32576.0,33442.0,34838.0,35627.0,35376.0,37985.0,39618.0,42109.0,43870,45321
Meat,155156148594,63666.0,2516.40,117.60120,46.299685,0.0,0.0,453.60,156.600,232.20,178.20,108.0,108.0,5.400,88452.0,26974,458,302760,556094,2367.88,2445.78,1720.0,1644.0,1592.0,1644.0,1762.0,1904.0,1996.0,2140.0,2192.0,2162.0,2236.0,2164.0,2282.0,2374.0,2494.0,2678.0,2916.0,3034.0,3090.0,3356.0,3550.0,3894.0,4022.0,3958.0,4118.0,3972.0,4250.0,4416.0,4618.0,4964.0,4994.0,4994.0,5158.0,5264.0,5492.0,5532.0,5686.0,6146.0,6148.0,6140.0,5818.0,6674.0,6988.0,7400.0,7802.0,7628.0,7948.0,8428.0,8522.0,9274.0,10194.0,10910,11368
Other,104179720300,57912.0,358.84,40.37075,15.893001,4.0,0.0,626.04,683.256,1746.32,135.24,444.0,272.0,79.656,33856.0,18972,288,210816,370224,1654.56,1933.38,0.0,0.0,0.0,0.0,0.0,18.0,18.0,18.0,18.0,36.0,36.0,18.0,18.0,36.0,36.0,54.0,72.0,72.0,108.0,126.0,126.0,126.0,108.0,126.0,126.0,126.0,126.0,126.0,198.0,144.0,126.0,54.0,90.0,72.0,72.0,18.0,36.0,18.0,54.0,36.0,54.0,36.0,54.0,72.0,54.0,72.0,144.0,198.0,288.0,288.0,468.0,468,522
Plant,49181785205,78505.0,249.90,33.33750,13.125000,0.0,0.0,1124.90,0.000,1999.90,124.95,280.0,280.0,126.000,35280.0,10013,183,92504,183002,709.35,-21.21,3396.0,3031.0,3323.0,3235.0,3494.0,3514.0,3533.0,3570.0,3847.0,4075.0,4216.0,4212.0,4420.0,4385.0,4523.0,4571.0,4295.0,4436.0,4575.0,4389.0,4351.0,4362.0,4743.0,4721.0,5590.0,5509.0,5561.0,5614.0,6039.0,5746.0,6423.0,6892.0,6686.0,7271.0,8465.0,8289.0,7925.0,9299.0,9194.0,11772.0,13435.0,16394.0,18479.0,21598.0,22539.0,23419.0,23408.0,24637.0,28826.0,29096.0,30153.0,27268,36581


In [30]:
# average of merged data frames grouped by normalized category
ctgry_grp_avg = fao_ofd_df.groupby('normalized_category').mean()
ctgry_grp_avg.head()

,created_t,energy_100g,proteins_100g,salt_100g,sodium_100g,ingredients_that_may_be_from_palm_oil_n,ingredients_from_palm_oil_n,fat_100g,sugars_100g,carbohydrates_100g,saturated-fat_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,fiber_100g,OFD_ID,FAO_ID,Area Code,Item Code,Element Code,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
normalized_category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Dairy,1.488619e+09,1720.000000,22.900000,1.380000,0.543307,0.000000,0.0,34.400,1.100000,1.900000,21.200000,15.000000,20.000000,1.200000,943.000000,59.400000,2.000000,2825.400000,5293.600000,33.940000,67.710000,119.600000,120.400000,134.600000,137.600000,151.200000,157.800000,171.200000,179.400000,181.000000,156.800000,143.000000,148.400000,164.800000,173.400000,177.400000,182.600000,166.800000,169.400000,171.600000,177.200000,182.200000,181.200000,192.600000,187.600000,152.000000,102.600000,116.600000,120.400000,124.000000,175.800000,187.000000,192.800000,219.400000,261.400000,289.600000,330.800000,363.000000,380.000000,433.600000,350.200000,233.200000,398.200000,380.400000,385.200000,371.400000,354.400000,377.600000,370.000000,378.400000,389.800000,388.000000,435.200000,440.200000
Grain,1.491551e+09,1155.000000,6.600000,0.550000,0.216535,0.000000,0.0,9.700,15.100000,39.800000,2.800000,4.000000,4.000000,1.500000,840.000000,211.141026,4.397436,2621.333333,5297.487179,24.240256,13.238974,138.012821,139.935897,133.615385,143.653846,148.602564,144.269231,157.961538,161.679487,167.435897,155.679487,160.500000,178.846154,184.923077,191.256410,205.525641,214.705128,210.833333,230.512821,235.756410,236.141026,239.410256,244.923077,244.833333,245.910256,256.846154,272.910256,280.243590,274.884615,298.205128,283.987179,275.743590,285.230769,301.371795,312.666667,307.358974,320.435897,319.987179,344.743590,354.076923,345.423077,365.282051,386.115385,401.679487,417.641026,428.743590,446.641026,456.756410,453.538462,486.987179,507.923077,539.858974,562.435897,581.038462
Meat,1.436631e+09,589.500000,23.300000,1.088900,0.428701,0.000000,0.0,4.200,1.450000,2.150000,1.650000,1.000000,1.000000,0.050000,819.000000,249.759259,4.240741,2803.333333,5149.018519,21.924815,22.646111,15.925926,15.222222,14.740741,15.222222,16.314815,17.629630,18.481481,19.814815,20.296296,20.018519,20.703704,20.037037,21.129630,21.981481,23.092593,24.796296,27.000000,28.092593,28.611111,31.074074,32.870370,36.055556,37.240741,36.648148,38.129630,36.777778,39.351852,40.888889,42.759259,45.962963,46.240741,46.240741,47.759259,48.740741,50.851852,51.222222,52.648148,56.907407,56.925926,56.851852,53.870370,61.796296,64.703704,68.518519,72.240741,70.629630,73.592593,78.037037,78.907407,85.870370,94.388889,101.018519,105.259259
Other,1.446941e+09,804.333333,4.983889,0.560705,0.220736,0.055556,0.0,8.695,9.489667,24.254444,1.878333,6.166667,3.777778,1.106333,470.222222,263.500000,4.000000,2928.000000,5142.000000,22.980000,26.852500,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.250000,0.250000,0.250000,0.500000,0.500000,0.250000,0.250000,0.500000,0.500000,0.750000,1.000000,1.000000,1.500000,1.750000,1.750000,1.750000,1.500000,1.750000,1.750000,1.750000,1.750000,1.750000,2.750000,2.000000,1.750000,0.750000,1.250000,1.000000,1.000000,0.250000,0.500000,0.250000,0.750000,0.500000,0.750000,0.500000,0.750000,1.000000,0.750000,1.000000,2.000000,2.750000,4.000000,4.000000,6.500000,6.500000,7.250000
Plant,1.405194e+09,2243.000000,7.140000,0.952500,0.375000,0.000000,0.0,32.140,0.000000,57.140000,3.570000,8.000000,8.000000,3.600000,1008.000000,286.085714,5.228571,2642.971429,5228.628571,20.267143,-0.606000,97.028571,86.600000,94.942857,92.428571,99.828571,100.400000,100

In [31]:
# sum of merged data frames grouped by FAO countries
# sorted by UK nutrition score per 100g
cntry_fao_grp_sum = fao_ofd_df.groupby('Area').sum().sort_values('nutrition-score-uk_100g', ascending=False)
cntry_fao_grp_sum.head()

,created_t,energy_100g,proteins_100g,salt_100g,sodium_100g,ingredients_that_may_be_from_palm_oil_n,ingredients_from_palm_oil_n,fat_100g,sugars_100g,carbohydrates_100g,saturated-fat_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,fiber_100g,OFD_ID,FAO_ID,Area Code,Item Code,Element Code,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
Area,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,326372100743,339671.0,1764.10,147.517174,58.079308,36.0,7.0,3578.49,6109.414,10271.86,1521.45,3044.0,3026.0,429.514,181350.0,7848,448,607792,1180612,7602.56,15167.04,11901.0,11667.0,11431.0,12564.0,12319.0,12143.0,13686.0,14803.0,14621.0,12623.0,11901.0,12308.0,13735.0,13996.0,13856.0,15630.0,14832.0,15579.0,16166.0,17368.0,15271.0,15294.0,15725.0,15179.0,12681.0,12216.0,12198.0,13400.0,12548.0,11691.0,10666.0,11095.0,12011.0,12384.0,12967.0,13289.0,14247.0,14874.0,15117.0,14671.0,14152.0,16314.0,19191.0,21335.0,22467.0,24996.0,26768.0,28034.0,31755.0,31217.0,31954.0,32485,33323
Albania,324010541496,324124.0,1793.05,162.914376,64.140923,31.0,6.0,3364.77,5567.214,9833.14,1294.84,2816.0,2773.0,412.214,180521.0,31630,669,607812,1169785,9176.45,4497.91,1909.0,2068.0,2125.0,2480.0,2184.0,2641.0,2565.0,2647.0,2763.0,2895.0,2913.0,3231.0,3383.0,3516.0,3464.0,3685.0,3708.0,4098.0,4053.0,4047.0,4284.0,4538.0,4723.0,4708.0,4581.0,4901.0,4899.0,5078.0,5521.0,5363.0,5039.0,5249.0,6811.0,8492.0,5832.0,6184.0,6141.0,5633.0,5778.0,6157.0,6270.0,6602.0,6270.0,5642.0,5856.0,6369.0,6794.0,8377.0,9244.0,9281.0,9798.0,10209,10379
Algeria,322605347633,321881.0,1785.91,161.961876,63.765923,31.0,6.0,3332.63,5567.214,9776.00,1291.27,2808.0,2765.0,408.614,179513.0,59053,888,605228,1164264,6222.66,368.52,13332.0,13754.0,13354.0,13942.0,13915.0,14381.0,15446.0,16447.0,16263.0,15538.0,18316.0,18729.0,18781.0,20523.0,24992.0,26960.0,29522.0,31267.0,36012.0,37739.0,38613.0,38513.0,42182.0,43575.0,46263.0,49812.0,53740.0,49674.0,55014.0,53715.0,53464.0,52952.0,54959.0,54183.0,54917.0,57630.0,58972.0,49204.0,53445.0,64067.0,64545.0,68620.0,72445.0,74709.0,70975.0,73700.0,78913.0,75198.0,83179.0,85742.0,82722.0,90826,89848
Angola,316436072527,315700.0,1798.41,162.524876,63.987576,31.0,6.0,3280.03,5479.614,9516.90,1276.37,2797.0,2754.0,398.714,176044.0,83218,1526,595449,1142559,-2441.60,3895.66,5828.0,6319.0,6925.0,6626.0,6919.0,7029.0,7262.0,7646.0,7740.0,8578.0,9048.0,8982.0,9061.0,8746.0,8191.0,9632.0,10051.0,10296.0,10180.0,10275.0,9461.0,9572.0,9347.0,9506.0,10066.0,10276.0,10465.0,10447.0,11343.0,11664.0,11673.0,11832.0,11359.0,11675.0,12789.0,13681.0,15027.0,16003.0,17844.0,18730.0,21950.0,26509.0,29208.0,32335.0,32946.0,34775.0,36968.0,39332.0,41801.0,42631.0,45242.0,43586,54648
Antigua and Barbuda,164586924751,191965.0,717.44,70.125992,27.609761,20.0,4.0,2048.02,3391.300,6039.52,801.56,1676.0,1676.0,256.900,95932.0,51612,904,292742,591279,1927.78,-6983.40,539.0,562.0,677.0,540.0,402.0,262.0,191.0,99.0,169.0,192.0,330.0,54.0,54.0,55.0,52.0,52.0,54.0,54.0,54.0,101.0,99.0,76.0,53.0,53.0,52.0,52.0,52.0,53.0,54.0,54.0,53.0,53.0,54.0,54.0,54.0,53.0,54.0,55.0,54.0,54.0,54.0,55.0,55.0,78.0,56.0,56.0,56.0,55.0,55.0,56.0,56.0,56,80


In [32]:
# average of merged data frames grouped by FAO countries
# sorted by UK nutrition score per 100g
cntry_fao_grp_avg = fao_ofd_df.groupby('Area').mean().sort_values('nutrition-score-uk_100g', ascending=False)
cntry_fao_grp_avg.head()

,created_t,energy_100g,proteins_100g,salt_100g,sodium_100g,ingredients_that_may_be_from_palm_oil_n,ingredients_from_palm_oil_n,fat_100g,sugars_100g,carbohydrates_100g,saturated-fat_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,fiber_100g,OFD_ID,FAO_ID,Area Code,Item Code,Element Code,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
Area,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Antigua and Barbuda,1.456521e+09,1698.805310,6.349027,0.620584,0.244334,0.176991,0.035398,18.124071,30.011504,53.447080,7.093451,14.831858,14.831858,2.273451,848.955752,456.743363,8.0,2590.637168,5232.557522,17.06,-61.80,4.769912,4.973451,5.991150,4.778761,3.557522,2.318584,1.690265,0.876106,1.495575,1.699115,2.920354,0.477876,0.477876,0.486726,0.460177,0.460177,0.477876,0.477876,0.477876,0.893805,0.876106,0.672566,0.469027,0.469027,0.460177,0.460177,0.460177,0.469027,0.477876,0.477876,0.469027,0.469027,0.477876,0.477876,0.477876,0.469027,0.477876,0.486726,0.477876,0.477876,0.477876,0.486726,0.486726,0.690265,0.495575,0.495575,0.495575,0.486726,0.486726,0.495575,0.495575,0.495575,0.707965
Afghanistan,1.457018e+09,1516.388393,7.875446,0.658559,0.259283,0.160714,0.031250,15.975402,27.274170,45.856518,6.792187,13.589286,13.508929,1.917473,809.598214,35.035714,2.0,2713.357143,5270.589286,33.94,67.71,53.129464,52.084821,51.031250,56.089286,54.995536,54.209821,61.098214,66.084821,65.272321,56.352679,53.129464,54.946429,61.316964,62.482143,61.857143,69.776786,66.214286,69.549107,72.169643,77.535714,68.174107,68.276786,70.200893,67.763393,56.611607,54.535714,54.455357,59.821429,56.017857,52.191964,47.616071,49.531250,53.620536,55.285714,57.888393,59.325893,63.602679,66.401786,67.486607,65.495536,63.178571,72.830357,85.674107,95.245536,100.299107,111.589286,119.500000,125.151786,141.763393,139.361607,142.651786,145.022321,148.763393
Angola,1.451542e+09,1448.165138,8.249587,0.745527,0.293521,0.142202,0.027523,15.046009,25.135844,43.655505,5.854908,12.830275,12.633028,1.828963,807.541284,381.733945,7.0,2731.417431,5241.096330,-11.20,17.87,26.733945,28.986239,31.766055,30.394495,31.738532,32.243119,33.311927,35.073394,35.504587,39.348624,41.504587,41.201835,41.564220,40.119266,37.573394,44.183486,46.105505,47.229358,46.697248,47.133028,43.399083,43.908257,42.876147,43.605505,46.174312,47.137615,48.004587,47.922018,52.032110,53.504587,53.545872,54.275229,52.105505,53.555046,58.665138,62.756881,68.931193,73.408257,81.853211,85.917431,100.688073,121.600917,133.981651,148.325688,151.128440,159.518349,169.577982,180.422018,191.747706,195.555046,207.532110,199.935780,250.678899
Algeria,1.453177e+09,1449.914414,8.044640,0.729558,0.287234,0.139640,0.027027,15.011847,25.077541,44.036036,5.816532,12.648649,12.454955,1.840604,808.617117,266.004505,4.0,2726.252252,5244.432432,28.03,1.66,60.054054,61.954955,60.153153,62.801802,62.680180,64.779279,69.576577,74.085586,73.256757,69.990991,82.504505,84.364865,84.599099,92.445946,112.576577,121.441441,132.981982,140.842342,162.216216,169.995495,173.932432,173.481982,190.009009,196.283784,208.391892,224.378378,242.072072,223.756757,247.810811,241.959459,240.828829,238.522523,247.563063,244.067568,247.373874,259.594595,265.639640,221.639640,240.743243,288.590090,290.743243,309.099099,326.328829,336.527027,319.707207,331.981982,355.463964,338.729730,374.680180,386.225225,372.621622,409.126126,404.720721
Albania,1.452962e+09,1453.470852,8.040583,0.730558,0.287627,0.139013,0.026906,15.088655,24.965085,44.094798,5.806457,12.627803,12.434978,1.848493,809.511211,141.838565,3.0,2725.614350,5245.672646,41.15,20.17,8.560538,9.273543,9.529148,11.121076,9.793722,11.843049,11.502242,11.869955,12.390135,12.982063,13.062780,14.

In [33]:
# sum of merged data frames grouped by OFD countries
# sorted by french nutrition score per 100g
cntry_ofd_grp_sum = fao_ofd_df.groupby('countries_en').sum().sort_values('nutrition-score-fr_100g', ascending=False)
cntry_ofd_grp_sum.head()

,created_t,energy_100g,proteins_100g,salt_100g,sodium_100g,ingredients_that_may_be_from_palm_oil_n,ingredients_from_palm_oil_n,fat_100g,sugars_100g,carbohydrates_100g,saturated-fat_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,fiber_100g,OFD_ID,FAO_ID,Area Code,Item Code,Element Code,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
countries_en,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
France,640869605036,716160.0,2738.8,273.488000,107.672233,149.0,29.0,8109.8,11702.3,21386.6,2738.3,6260.0,6077.0,830.2,356226.0,97061,1911,1166283,2286204,9986.50,7345.14,19437.0,20141.0,19884.0,21078.0,20949.0,21048.0,22854.0,24131.0,24170.0,22689.0,24041.0,25238.0,26190.0,27263.0,29551.0,32369.0,33173.0,35359.0,37962.0,39425.0,38572.0,38656.0,40337.0,40778.0,40991.0,43394.0,45444.0,43769.0,47376.0,46029.0,44719.0,45038.0,47689.0,48651.0,47751.0,50515.0,52398.0,48389.0,52030.0,55627.0,57039.0,61813.0,65608.0,67875.0,66816.0,70742.0,75573.0,75308.0,81849.0,83660.0,84352.0,89968,91380
"France,United Kingdom",354163714115,358086.0,2618.4,287.852000,113.327559,0.0,0.0,3785.2,6787.9,10084.3,2199.9,4062.0,4062.0,418.9,164154.0,59691,1090,673420,1286914,5388.72,4392.25,4678.0,4888.0,4916.0,5172.0,4941.0,5209.0,5605.0,6109.0,5947.0,5765.0,6252.0,6059.0,6298.0,6693.0,7191.0,8314.0,9119.0,9454.0,10462.0,11221.0,10934.0,11149.0,12024.0,12383.0,12137.0,12613.0,13573.0,13222.0,13783.0,13579.0,13067.0,12859.0,13540.0,13618.0,13380.0,14077.0,14965.0,12534.0,13779.0,16067.0,16158.0,17649.0,18980.0,19681.0,18982.0,20193.0,22310.0,22582.0,24358.0,24717.0,24473.0,26471,26668
"Belgium,France,Netherlands,United Kingdom",86641234678,109852.0,353.8,40.890000,16.098425,0.0,0.0,988.9,1957.5,3880.2,600.3,1102.0,1102.0,168.2,47850.0,13066,260,155272,305816,1307.40,929.94,1468.0,1568.0,1598.0,1670.0,1564.0,1636.0,1758.0,1930.0,1854.0,1772.0,1962.0,1916.0,1988.0,2084.0,2294.0,2670.0,2884.0,3000.0,3394.0,3648.0,3442.0,3380.0,3678.0,3746.0,3662.0,3916.0,4172.0,3940.0,4256.0,4170.0,4040.0,3964.0,4190.0,4168.0,4054.0,4338.0,4656.0,3562.0,4040.0,4894.0,4974.0,5398.0,5882.0,6062.0,5726.0,6200.0,6908.0,6908.0,7608.0,7630.0,7290.0,7948,7932
Germany,80542973042,56434.0,145.0,5.892800,2.320000,0.0,0.0,0.0,2363.5,3161.0,0.0,580.0,580.0,0.0,47850.0,13066,260,155272,305816,1307.40,929.94,1468.0,1568.0,1598.0,1670.0,1564.0,1636.0,1758.0,1930.0,1854.0,1772.0,1962.0,1916.0,1988.0,2084.0,2294.0,2670.0,2884.0,3000.0,3394.0,3648.0,3442.0,3380.0,3678.0,3746.0,3662.0,3916.0,4172.0,3940.0,4256.0,4170.0,4040.0,3964.0,4190.0,4168.0,4054.0,4338.0,4656.0,3562.0,4040.0,4894.0,4974.0,5398.0,5882.0,6062.0,5726.0,6200.0,6908.0,6908.0,7608.0,7630.0,7290.0,7948,7932
United Kingdom,170323135514,127514.0,1459.2,21.278272,8.377273,0.0,0.0,1124.8,2560.0,3456.4,390.4,496.0,532.0,287.0,136075.0,28661,521,330076,624999,2675.18,2367.65,2328.0,2390.0,2394.0,2492.0,2445.0,2590.0,2758.0,3002.0,2952.0,2857.0,3084.0,3000.0,3131.0,3275.0,3545.0,4015.0,4350.0,4525.0,4951.0,5340.0,5231.0,5341.0,5701.0,5739.0,5735.0,5916.0,6311.0,6162.0,6587.0,6668.0,6551.0,6467.0,6779.0,6808.0,6808.0,7106.0,7503.0,6637.0,7120.0,7968.0,7889.0,8739.0,9382.0,9770.0,9633.0,10022.0,10898.0,11144.0,11901.0,12299.0,12439.0,13455,13674


In [34]:
# average of merged data frames grouped by OFD countries
cntry_ofd_grp_avg = fao_ofd_df.groupby('countries_en').mean()
cntry_ofd_grp_avg.head()

,created_t,energy_100g,proteins_100g,salt_100g,sodium_100g,ingredients_that_may_be_from_palm_oil_n,ingredients_from_palm_oil_n,fat_100g,sugars_100g,carbohydrates_100g,saturated-fat_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,fiber_100g,OFD_ID,FAO_ID,Area Code,Item Code,Element Code,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
countries_en,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Belgium,France,Netherlands,United Kingdom",1.493814e+09,1894.000000,6.100000,0.705000,0.277559,0.000000,0.00000,17.050000,33.750000,66.900000,10.350000,19.000000,19.000000,2.900000,825.000000,225.275862,4.482759,2677.103448,5272.689655,22.541379,16.033448,25.310345,27.034483,27.551724,28.793103,26.965517,28.206897,30.310345,33.275862,31.965517,30.551724,33.827586,33.034483,34.275862,35.931034,39.551724,46.034483,49.724138,51.724138,58.517241,62.896552,59.344828,58.275862,63.413793,64.586207,63.137931,67.517241,71.931034,67.931034,73.379310,71.896552,69.655172,68.344828,72.241379,71.862069,69.896552,74.793103,80.275862,61.413793,69.655172,84.379310,85.758621,93.068966,101.413793,104.517241,98.724138,106.896552,119.103448,119.103448,131.172414,131.551724,125.689655,137.034483,136.758621
Canada,1.485317e+09,1036.810811,5.567838,0.794568,0.312946,0.000000,0.00000,10.417838,19.436649,33.668649,2.633514,7.918919,7.918919,1.822378,677.000000,233.540541,4.378378,2731.351351,5244.432432,22.636216,18.372703,19.837838,21.189189,21.594595,22.567568,21.135135,22.162162,23.810811,26.135135,25.108108,24.054054,26.621622,25.945946,26.918919,28.270270,31.108108,36.243243,39.189189,40.756757,46.189189,49.675676,46.891892,46.054054,50.027027,51.000000,49.864865,53.297297,56.756757,53.621622,58.108108,56.783784,54.972973,53.729730,56.891892,56.540541,55.000000,58.675676,63.027027,48.189189,54.756757,66.243243,67.378378,73.054054,79.648649,82.135135,77.540541,84.000000,93.783784,93.945946,103.675676,103.972973,99.918919,108.810811,108.756757
France,1.476658e+09,1650.138249,6.310599,0.630157,0.248093,0.343318,0.06682,18.686175,26.963825,49.277880,6.309447,14.423963,14.002304,1.912903,820.797235,223.642857,4.403226,2687.288018,5267.751152,23.010369,16.924286,44.785714,46.407834,45.815668,48.566820,48.269585,48.497696,52.658986,55.601382,55.691244,52.278802,55.394009,58.152074,60.345622,62.817972,68.089862,74.582949,76.435484,81.472350,87.470046,90.841014,88.875576,89.069124,92.942396,93.958525,94.449309,99.986175,104.709677,100.850230,109.161290,106.057604,103.039171,103.774194,109.882488,112.099078,110.025346,116.394009,120.732719,111.495392,119.884793,128.172811,131.426267,142.426267,151.170507,156.394009,153.953917,163.000000,174.131336,173.520737,188.592166,192.764977,194.359447,207.299539,210.552995
"France,United Kingdom",1.439690e+09,1455.634146,10.643902,1.170130,0.460681,0.000000,0.00000,15.386992,27.593089,40.993089,8.942683,16.512195,16.512195,1.702846,667.292683,242.646341,4.430894,2737.479675,5231.357724,21.905366,17.854675,19.016260,19.869919,19.983740,21.024390,20.085366,21.174797,22.784553,24.833333,24.174797,23.434959,25.414634,24.630081,25.601626,27.207317,29.231707,33.796748,37.069106,38.430894,42.528455,45.613821,44.447154,45.321138,48.878049,50.337398,49.337398,51.272358,55.174797,53.747967,56.028455,55.199187,53.117886,52.272358,55.040650,55.357724,54.390244,57.223577,60.833333,50.951220,56.012195,65.313008,65.682927,71.743902,77.154472,80.004065,77.162602,82.085366,90.691057,91.796748,99.016260,100.475610,99.483740,107.605691,108.406504
Germany,1.388672e+09,973.000000,2.500000,0.101600,0.040000,0.000000,0.00000,0.000000,40.750000,54.500000,0.000000,10.000000,10.000000,0.000000,825.000000,225.275862,4.482759,2677.103448,5272

In [35]:
# sum of merged data frames grouped by normalized category, and OFD countries
ctgry_ofd_cntry_grp_sum = fao_ofd_df.groupby(['normalized_category', 'countries_en']).sum()
ctgry_ofd_cntry_grp_sum.head()

created_t  energy_100g  \
normalized_category countries_en                                       
Dairy               France                   7443095245       8600.0   
Grain               France                 116341009902      90090.0   
Meat                France,United Kingdom   78496777674      30456.0   
                    United Kingdom          76659370920      33210.0   
Other               Canada                  11906197140       6752.0   

                                           proteins_100g  salt_100g  \
normalized_category countries_en                                      
Dairy               France                        114.50     6.9000   
Grain               France                        514.80    42.9000   
Meat                France,United Kingdom        1323.00   108.0000   
                    United Kingdom               1193.40     9.6012   
Other               Canada                        109.44    10.6360   

                                           sodium_100g  \
normalized_category countries_en                         
Dairy               France                    2.716535   
Grain               France                   16.889764   
Meat                France,United Kingdom    42.519685   
                    United Kingdom            3.780000   
Other               Canada                    4.184000   

                                           ingredients_that_may_be_from_palm_oil_n  \
normalized_category countries_en                                                     
Dairy               France                                                     0.0   
Grain               France                                                     0.0   
Meat                France,United Kingdom                                      0.0   
                    United Kingdom                                             0.0   
Other               Canada                                                     0.0   

                                           ingredients_from_palm_oil_n  \
normalized_category countries_en                                         
Dairy               France                                         0.0   
Grain               France                                         0.0   
Meat                France,United Kingdom                          0.0   
                    United Kingdom                                 0.0   
Other               Canada                                         0.0   

                                           fat_100g  sugars_100g  \
normalized_category countries_en                                   
Dairy               France                   172.00        5.500   
Grain               France                   756.60     1177.800   
Meat                France,United Kingdom    108.00      156.600   
                    United Kingdom           345.60        0.000   
Other               Canada                    75.16        2.856   

                                           carbohydrates_100g  \
normalized_category countries_en                                
Dairy               France                               9.50   
Grain               France                            3104.40   
Meat                France,United Kingdom              232.20   
                    United Kingdom                       0.00   
Other               Canada                             123.44   

                                           saturated-fat_100g  \
normalized_category countries_en                                
Dairy               France                             106.00   
Grain               France                             218.40   
Meat                France,United Kingdom               37.80   
                    United Kingdom                     140.40   
Other               Canada                              39.44   

                                           nutrition-score-fr_100g  \
normalized_category countries_en                       

In [36]:
# average of merged data frames grouped by normalized category, and OFD countries
# sorted by energy per 100g serving
ctgry_ofd_cntry_grp_avg = fao_ofd_df.groupby(['normalized_category', 'countries_en']).mean().sort_values('energy_100g', ascending=False)
ctgry_ofd_cntry_grp_avg.head()

created_t  \
normalized_category countries_en                                              
Other               United Kingdom                             1.422157e+09   
Plant               United States                              1.405194e+09   
Sweet               France,United Kingdom                      1.445631e+09   
                    France                                     1.475330e+09   
                    Belgium,France,Netherlands,United Kingdom  1.493814e+09   

                                                               energy_100g  \
normalized_category countries_en                                             
Other               United Kingdom                                  2479.0   
Plant               United States                                   2243.0   
Sweet               France,United Kingdom                           2018.2   
                    France                                          1894.0   
                    Belgium,France,Netherlands,United Kingdom       1894.0   

                                                               proteins_100g  \
normalized_category countries_en                                               
Other               United Kingdom                                  4.950000   
Plant               United States                                   7.140000   
Sweet               France,United Kingdom                           5.580000   
                    France                                          6.427273   
                    Belgium,France,Netherlands,United Kingdom       6.100000   

                                                               salt_100g  \
normalized_category countries_en                                           
Other               United Kingdom                              0.010004   
Plant               United States                               0.952500   
Sweet               France,United Kingdom                       0.660000   
                    France                                      0.667273   
                    Belgium,France,Netherlands,United Kingdom   0.705000   

                                                               sodium_100g  \
normalized_category countries_en                                             
Other               United Kingdom                                0.003939   
Plant               United States                                 0.375000   
Sweet               France,United Kingdom                         0.259843   
                    France                                        0.262706   
                    Belgium,France,Netherlands,United Kingdom     0.277559   

                                                               ingredients_that_may_be_from_palm_oil_n  \
normalized_category countries_en                                                                         
Other               United Kingdom                                                            0.000000   
Plant               United States                                                             0.000000   
Sweet               France,United Kingdom                                                     0.000000   
                    France                                                                    0.454545   
                    Belgium,France,Netherlands,United Kingdom                                 0.000000   

                                                               ingredients_from_palm_oil_n  \
normalized_category countries_en                                                             
Other               United Kingdom                                                0.000000   
Plant               United States                                                 0.000000   
Sweet               France,United Kingdom                                         0.000000   
                    France                                                        0.090909   
                 

In [37]:
# average of merged data frames grouped by normalized category, and FAO countries
ctgry_fao_cntry_grp_sum = fao_ofd_df.groupby(['normalized_category', 'Area']).sum()
ctgry_fao_cntry_grp_sum.head()

created_t  energy_100g  proteins_100g  \
normalized_category Area                                                   
Dairy               Afghanistan   7443095245       8600.0          114.5   
Grain               Afghanistan  19390168317      15015.0           85.8   
                    Albania      31322579589      24255.0          138.6   
                    Algeria      31322579589      24255.0          138.6   
                    Angola       20881719726      16170.0           92.4   

                                 salt_100g  sodium_100g  \
normalized_category Area                                  
Dairy               Afghanistan       6.90     2.716535   
Grain               Afghanistan       7.15     2.814961   
                    Albania          11.55     4.547244   
                    Algeria          11.55     4.547244   
                    Angola            7.70     3.031496   

                                 ingredients_that_may_be_from_palm_oil_n  \
normalized_category Area                                                   
Dairy               Afghanistan                                      0.0   
Grain               Afghanistan                                      0.0   
                    Albania                                          0.0   
                    Algeria                                          0.0   
                    Angola                                           0.0   

                                 ingredients_from_palm_oil_n  fat_100g  \
normalized_category Area                                                 
Dairy               Afghanistan                          0.0     172.0   
Grain               Afghanistan                          0.0     126.1   
                    Albania                              0.0     203.7   
                    Algeria                              0.0     203.7   
                    Angola                               0.0     135.8   

                                 sugars_100g  carbohydrates_100g  \
normalized_category Area                                           
Dairy               Afghanistan          5.5                 9.5   
Grain               Afghanistan        196.3               517.4   
                    Albania            317.1               835.8   
                    Algeria            317.1               835.8   
                    Angola             211.4               557.2   

                                 saturated-fat_100g  nutrition-score-fr_100g  \
normalized_category Area                                                       
Dairy               Afghanistan               106.0                     75.0   
Grain               Afghanistan                36.4                     52.0   
                    Albania                    58.8                     84.0   
                    Algeria                    58.8                     84.0   
                    Angola                     39.2                     56.0   

                                 nutrition-score-uk_100g  fiber_100g   OFD_ID  \
normalized_category Area                                                        
Dairy               Afghanistan                    100.0         6.0   4715.0   
Grain               Afghanistan                     52.0        19.5  10920.0   
                    Albania                         84.0        31.5  17640.0   
                    Algeria                         84.0        31.5  17640.0   
                    Angola                          56.0        21.0  11760.0   

                                 FAO_ID  Area Code  Item Code  Element Code  \
normalized_category Area                                                      
Dairy               Afghanistan     297         10      14127         26468   
Grain               Afghanistan     302         26      34617         68741   
                    Albania        2319         63      55090        112151   
                    Algeria        

In [38]:
# average of merged data frames grouped by normalized category, and FAO countries
# double sorted by highest protein and area in reverse alphabetical
ctgry_fao_cntry_grp_avg = fao_ofd_df.groupby(['normalized_category', 'Area']).mean().sort_values(['proteins_100g', 'Area'], ascending=[False, False])
ctgry_fao_cntry_grp_avg

created_t  energy_100g  \
normalized_category Area                                             
Meat                Angola               1.436631e+09   589.500000   
                    Algeria              1.436631e+09   589.500000   
                    Albania              1.436631e+09   589.500000   
                    Afghanistan          1.436631e+09   589.500000   
                    Antigua and Barbuda  1.436631e+09   589.500000   
Dairy               Afghanistan          1.488619e+09  1720.000000   
Seafood             Afghanistan          1.398323e+09   725.000000   
                    Angola               1.398323e+09   725.000000   
                    Algeria              1.398323e+09   725.000000   
                    Albania              1.398323e+09   725.000000   
Plant               Antigua and Barbuda  1.405194e+09  2243.000000   
                    Angola               1.405194e+09  2243.000000   
                    Algeria              1.405194e+09  2243.000000   
                    Afghanistan          1.405194e+09  2243.000000   
                    Albania              1.405194e+09  2243.000000   
Grain               Antigua and Barbuda  1.491551e+09  1155.000000   
                    Angola               1.491551e+09  1155.000000   
                    Afghanistan          1.491551e+09  1155.000000   
                    Algeria              1.491551e+09  1155.000000   
                    Albania              1.491551e+09  1155.000000   
Sweet               Antigua and Barbuda  1.458423e+09  1752.913043   
                    Afghanistan          1.458423e+09  1752.913043   
                    Angola               1.458423e+09  1752.913043   
                    Algeria              1.458423e+09  1752.913043   
                    Albania              1.458423e+09  1752.913043   
Other               Angola               1.446941e+09   804.333333   
                    Algeria              1.446941e+09   804.333333   
                    Albania              1.446941e+09   804.333333   
                    Afghanistan          1.446941e+09   804.333333   

                                         proteins_100g  salt_100g  \
normalized_category Area                                            
Meat                Angola                   23.300000   1.088900   
                    Algeria                  23.300000   1.088900   
                    Albania                  23.300000   1.088900   
                    Afghanistan              23.300000   1.088900   
                    Antigua and Barbuda      23.300000   1.088900   
Dairy               Afghanistan              22.900000   1.380000   
Seafood             Afghanistan              12.100000   2.235200   
                    Angola                   12.100000   2.235200   
                    Algeria                  12.100000   2.235200   
                    Albania                  12.100000   2.235200   
Plant               Antigua and Barbuda       7.140000   0.952500   
                    Angola                    7.140000   0.952500   
                    Algeria                   7.140000   0.952500   
                    Afghanistan               7.140000   0.952500   
                    Albania                   7.140000   0.952500   
Grain               Antigua and Barbuda       6.600000   0.550000   
                    Angola                    6.600000   0.550000   
                    Afghanistan               6.600000   0.550000   
                    Algeria                   6.600000   0.550000   
                    Albania                   6.600000   0.550000   
Sweet               Antigua and Barbuda       5.518696   0.578265   
                    Afghanistan               5.518696   0.578265   
                    Angola                    5.518696   0.578265   
                    Algeria                   5.518696   0.578265   
                    Albania                   5.518696   0.578265   
Other     

In [39]:
# sum of merged data frames grouped by normalized category, and OFD brands
# sorted by sugars per 100g
ctgry_fao_brnd_grp_sum = fao_ofd_df.groupby(['normalized_category', 'brands']).sum().sort_values('sugars_100g', ascending=False)
ctgry_fao_brnd_grp_sum.head()

created_t  energy_100g  \
normalized_category brands                                               
Sweet               Marks & Spencer          382291617008     517766.0   
                    Bijou                    299261659501     366357.0   
                    Asda,Asda Extra Special   82286510346      74472.0   
                    Candy Crush               80542973042      56434.0   
                    Marks & Spencer,Disney    43213519585      65105.0   

                                             proteins_100g   salt_100g  \
normalized_category brands                                               
Sweet               Marks & Spencer                 1589.2  194.590000   
                    Bijou                           1249.9  132.530000   
                    Asda,Asda Extra Special          226.2   11.597042   
                    Candy Crush                      145.0    5.892800   
                    Marks & Spencer,Disney           197.2    5.800000   

                                             sodium_100g  \
normalized_category brands                                 
Sweet               Marks & Spencer            76.610236   
                    Bijou                      52.177165   
                    Asda,Asda Extra Special     4.565765   
                    Candy Crush                 2.320000   
                    Marks & Spencer,Disney      2.283465   

                                             ingredients_that_may_be_from_palm_oil_n  \
normalized_category brands                                                             
Sweet               Marks & Spencer                                              0.0   
                    Bijou                                                      145.0   
                    Asda,Asda Extra Special                                      0.0   
                    Candy Crush                                                  0.0   
                    Marks & Spencer,Disney                                       0.0   

                                             ingredients_from_palm_oil_n  \
normalized_category brands                                                 
Sweet               Marks & Spencer                                 29.0   
                    Bijou                                            0.0   
                    Asda,Asda Extra Special                          0.0   
                    Candy Crush                                      0.0   
                    Marks & Spencer,Disney                           0.0   

                                             fat_100g  sugars_100g  \
normalized_category brands                                           
Sweet               Marks & Spencer            5849.3       8726.1   
                    Bijou                      4118.0       6728.0   
                    Asda,Asda Extra Special     406.0       2552.0   
                    Candy Crush                   0.0       2363.5   
                    Marks & Spencer,Disney      942.5       1551.5   

                                             carbohydrates_100g  \
normalized_category brands                                        
Sweet               Marks & Spencer                     15790.5   
                    Bijou                               11194.0   
                    Asda,Asda Extra Special              3143.6   
                    Candy Crush                          3161.0   
                    Marks & Spencer,Disney               1557.3   

                                             saturated-fat_100g  \
normalized_category brands                                        
Sweet               Marks & Spencer                      3140.7   
                    Bijou                                 986.0   
                    Asda,Asda Extra Special               208.8   
                    Candy Crush                             0.0   
                    Marks & Spencer,Disney                582.9   

        

In [40]:
# average of merged data frames grouped by normalized category, and OFD brands
ctgry_fao_brnd_grp_avg = fao_ofd_df.groupby(['normalized_category', 'brands']).mean()
ctgry_fao_brnd_grp_avg.head()

created_t  \
normalized_category brands                                            
Dairy               Marks & Spencer,M&S                1.488619e+09   
Grain               Marks & Spencer,Marks and Spencer  1.491551e+09   
Meat                Marks & Spencer,M&S                1.453644e+09   
                    Simply M&S,Marks & Spencer         1.419618e+09   
Other               A&W                                1.420150e+09   

                                                       energy_100g  \
normalized_category brands                                           
Dairy               Marks & Spencer,M&S                     1720.0   
Grain               Marks & Spencer,Marks and Spencer       1155.0   
Meat                Marks & Spencer,M&S                      564.0   
                    Simply M&S,Marks & Spencer               615.0   
Other               A&W                                      215.0   

                                                       proteins_100g  \
normalized_category brands                                             
Dairy               Marks & Spencer,M&S                         22.9   
Grain               Marks & Spencer,Marks and Spencer            6.6   
Meat                Marks & Spencer,M&S                         24.5   
                    Simply M&S,Marks & Spencer                  22.1   
Other               A&W                                          0.0   

                                                       salt_100g  sodium_100g  \
normalized_category brands                                                      
Dairy               Marks & Spencer,M&S                   1.3800     0.543307   
Grain               Marks & Spencer,Marks and Spencer     0.5500     0.216535   
Meat                Marks & Spencer,M&S                   2.0000     0.787402   
                    Simply M&S,Marks & Spencer            0.1778     0.070000   
Other               A&W                                   0.0616     0.024200   

                                                       ingredients_that_may_be_from_palm_oil_n  \
normalized_category brands                                                                       
Dairy               Marks & Spencer,M&S                                                    0.0   
Grain               Marks & Spencer,Marks and Spencer                                      0.0   
Meat                Marks & Spencer,M&S                                                    0.0   
                    Simply M&S,Marks & Spencer                                             0.0   
Other               A&W                                                                    0.0   

                                                       ingredients_from_palm_oil_n  \
normalized_category brands                                                           
Dairy               Marks & Spencer,M&S                                        0.0   
Grain               Marks & Spencer,Marks and Spencer                          0.0   
Meat                Marks & Spencer,M&S                                        0.0   
                    Simply M&S,Marks & Spencer                                 0.0   
Other               A&W                                                        0.0   

                                                       fat_100g  sugars_100g  \
normalized_category brands                                                     
Dairy               Marks & Spencer,M&S                    34.4          1.1   
Grain               Marks & Spencer,Marks and Spencer       9.7         15.1   
Meat                Marks & Spencer,M&S                     2.0          2.9   
                    Simply M&S,Marks & Spencer              6.4          0.0   
Other               A&W                                     0.0         13.6   

                                                       carbohydrates_100g  \
normalized_category brands                                          

A few groupings of the original dataframes

In [41]:
# group FAO data by 'Area' (country) and 'Item' (category) 

cntry_item_grp = fao_data_df.groupby(['Area', 'FAO_Category']).sum()
cntry_item_grp.head()

ID  Area Code  Item Code  Element Code  \
Area        FAO_Category                                                  
Afghanistan Alcoholic Beverages  74          2       2924          5142   
            Animal fats          77          2       2946          5142   
            Apples and products  34          2       2617          5142   
            Bananas              33          2       2615          5142   
            Barley and products   5          4       5026         10663   

                                 latitude  longitude  Y1961  Y1962  Y1963  \
Area        FAO_Category                                                    
Afghanistan Alcoholic Beverages     33.94      67.71    0.0    0.0    0.0   
            Animal fats             33.94      67.71   20.0   20.0   21.0   
            Apples and products     33.94      67.71   14.0   14.0   14.0   
            Bananas                 33.94      67.71    0.0    0.0    0.0   
            Barley and products     67.88     135.42  313.0  313.0  313.0   

                                 Y1964  Y1965  Y1966  Y1967  Y1968  Y1969  \
Area        FAO_Category                                                    
Afghanistan Alcoholic Beverages    0.0    0.0    0.0    0.0    0.0    0.0   
            Animal fats           22.0   24.0   25.0   27.0   29.0   29.0   
            Apples and products   17.0   19.0   22.0   24.0   25.0   26.0   
            Bananas                0.0    0.0    0.0    0.0    0.0    0.0   
            Barley and products  314.0  314.0  312.0  296.0  299.0  303.0   

                                 Y1970  Y1971  Y1972  Y1973  Y1974  Y1975  \
Area        FAO_Category                                                    
Afghanistan Alcoholic Beverages    0.0    0.0    0.0    0.0    0.0    0.0   
            Animal fats           27.0   26.0   25.0   27.0   28.0   29.0   
            Apples and products   24.0   20.0   20.0   22.0   21.0   21.0   
            Bananas                0.0    0.0    0.0    0.0    0.0    0.0   
            Barley and products  308.0  294.0  289.0  297.0  316.0  321.0   

                                 Y1976  Y1977  Y1978  Y1979  Y1980  Y1981  \
Area        FAO_Category                                                    
Afghanistan Alcoholic Beverages    0.0    0.0    0.0    0.0    0.0    0.0   
            Animal fats           31.0   31.0   31.0   31.0   31.0   32.0   
            Apples and products   21.0   13.0   18.0   19.0   17.0   18.0   
            Bananas                0.0    0.0    1.0    1.0    1.0    9.0   
            Barley and products  335.0  245.0  268.0  262.0  266.0  249.0   

                                 Y1982  Y1983  Y1984  Y1985  Y1986  Y1987  \
Area        FAO_Category                                                    
Afghanistan Alcoholic Beverages    0.0    0.0    0.0    0.0    0.0    0.0   
            Animal fats           32.0   32.0   30.0   26.0   20.0   22.0   
            Apples and products   16.0   15.0   13.0   15.0   16.0   11.0   
            Bananas               11.0    4.0    3.0    4.0    5.0    9.0   
            Barley and products  229.0  220.0  211.0  199.0  191.0  191.0   

                                 Y1988  Y1989  Y1990  Y1991  Y1992  Y1993  \
Area        FAO_Category                                                    
Afghanistan Alcoholic Beverages    0.0    0.0    0.0    0.0    0.0    0.0   
            Animal fats           23.0   24.0   29.0   30.0   30.0   33.0   
            Apples and products   14.0   16.0   16.0   15.0   15.0   15.0   
            Bananas                6.0    0.0    0.0    1.0    4.0    3.0   
            Barley and products  195.0  191.0  178.0  175.0  160.0  205.0   

                                 Y1994  Y1995  Y1996  Y1997  Y1998  Y1999  \
Area        FAO_Category                                                    
Afghanistan Alcoholic Beverages    0.0    0.0    0.0    0.0    0.0    0.0   
            Animal fats           37

In [42]:
# drop NaN values, group by count of categories_en where count of products > 1, descending 
category_df = ofd_rel_df.dropna(axis=0, subset=['categories_en'])
grp_ctgry = category_df.groupby(['categories_en'])
grp_cnt = grp_ctgry.count().sort_values('created_t', ascending=False)
grp_cnt_df = pd.DataFrame(grp_cnt['created_t'])
grp_cnt_df = grp_cnt_df.rename(columns={'created_t': 'count'})
grp_cnt_df = grp_cnt_df.loc[grp_cnt_df['count'] > 1, :]
grp_cnt_df

,count
categories_en,
"Beverages,Non-sugared beverages",2345
"Sugary snacks,Biscuits and cakes,Biscuits",893
Fats,732
"Sugary snacks,Chocolates,Dark chocolates",678
"Sugary snacks,Confectioneries,Candies",587
"Groceries,Sauces,Tomato sauces,Ketchup",561
"Salty snacks,Appetizers,Crackers",534
"Dairies,Milks",504
"Sugary snacks,Biscuits and cakes,Biscuits,Chocolate biscuits",498


In [43]:
# drop NaN values, group b brands where count of products > 300, descending 

brand_df = ofd_rel_df.dropna(axis=0, subset=['brands'])
grp_brnd = brand_df.groupby(['brands'])
brnd_cnt = grp_brnd.count().sort_values('created_t', ascending=False)
brnd_cnt_df = pd.DataFrame(brnd_cnt['created_t'])
brnd_cnt_df = brnd_cnt_df.rename(columns={'created_t': 'count'})
brnd_cnt_df = brnd_cnt_df.loc[brnd_cnt_df['count'] > 300, :]
brnd_cnt_df

,count
brands,
Carrefour,3670
Auchan,3217
U,2608
Casino,2091
Leader Price,2015
Meijer,1995
Kroger,1662
Cora,1374
Ahold,1370


In [44]:
# drop NaN values, group by country where count of products > 300, descending 

country_df = ofd_rel_df.dropna(axis=0, subset=['countries_en'])
grp_cntry = country_df.groupby(['countries_en'])
cntry_cnt = grp_cntry.count().sort_values('created_t', ascending=False)
cntry_cnt_df = pd.DataFrame(cntry_cnt['created_t'])
cntry_cnt_df = cntry_cnt_df.rename(columns={'created_t': 'count'})
cntry_cnt_df = cntry_cnt_df.loc[cntry_cnt_df['count'] > 1, :]
cntry_cnt_df

,count
countries_en,
United States,173159
France,123961
Switzerland,14932
Germany,8470
Spain,5529
United Kingdom,5092
Belgium,2936
Australia,2201
Russia,1582
